***
#  Part 1: Data Retrieval
***

# Gather relevant data 

## Webscrape demographic data 

Import Relevant packages

In [1]:
import pandas as pd
import numpy as np
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

Checks to confirm page exists

In [2]:
url = 'https://www.zip-codes.com{}'
columbus_branch ='/city/oh-columbus.asp'
# reinforces integrity of the data collection
accessed = False
while accessed == False:
    try:
        response = requests.get(url.format(columbus_branch))
        print(response)
        accessed = True
    except:
        accesssed = False

<Response [200]>


Gathers relevant data on the page

In [3]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    tbl = soup.table.tr.find_all('td')[1].find_all('table')[1]
    print(tbl)

<table border="0" cellpadding="0" cellspacing="0" class="statTable" id="tblZIP" width="99%">
<tr><td class="label"><strong>ZIP Code</strong></td><td><strong>Type</strong></td><td><strong>County</strong></td><td><strong>Population</strong></td><td><strong>Area Code(s)</strong></td></tr>
<tr><td><a href="/zip-code/43081/zip-code-43081.asp" style="text-decoration:underline;" title="ZIP Code 43081, WESTERVILLE, OH">ZIP Code 43081</a></td><td>Standard</td><td><a href="/county/oh-franklin.asp">Franklin</a></td><td>55,991</td><td><a href="/area-code/area-code-614.asp" title="Area Code 614">614</a></td></tr><tr><td><a href="/zip-code/43085/zip-code-43085.asp" style="text-decoration:underline;" title="ZIP Code 43085, COLUMBUS, OH">ZIP Code 43085</a></td><td>Standard</td><td><a href="/county/oh-franklin.asp">Franklin</a></td><td>23,258</td><td><a href="/area-code/area-code-614.asp" title="Area Code 614">614</a> / <a href="/area-code/area-code-380.asp" title="Area Code 380">380</a> / <a href="/ar

function to help web scrape zip code information

In [4]:
# @TODO: Fix zip info extraction method.
# utilize when using the table the [0-4] table 
# BeautifulSoup(response.text, 'html.parser').table.tr.find_all('td')[1].find_all('table')
def zip_info(response):
    page = BeautifulSoup(response.text, 'html.parser').find_all("table", class_="statTable")
    #print(page)
    dict = {}
    for table in page:
        for row in table.find_all('tr'):
            dict[row.find_all('td')[0].string] = row.find_all('td')[1].string        
    return dict

Parses through and gathers data

In [5]:
i=0
zip_info_df = pd.DataFrame()
zip_url = '/zip-code/{}/zip-code-{}.asp'
for row in tbl.find_all('tr'):
    #checks for header
    if row.find_all('td')[0].string != 'ZIP Code':
        if row.find_all('td')[1].string == 'Standard':
            zip_code = row.find_all('td')[0].string.strip(" ZIPCode")
            print(zip_code)
            #coded just to make sure that we can access the website
            accessed = False
            while accessed == False:
                try:
                    zip_response = requests.get(url.format(zip_url.format(zip_code,zip_code)))
                    accessed = True
                    i+=1
                    zip_dict = zip_info(zip_response)
                    zip_info_df = zip_info_df.append(zip_dict, ignore_index=True)
                except:
                    accessed = False
print(i)
print(zip_info_df)
# reads in all zip code responses

43081
43085
43201
43202
43203
43204
43205
43206
43207
43209
43210
43211
43212
43213
43214
43215
43217
43219
43220
43221
43222
43223
43224
43227
43228
43229
43230
43231
43232
43235
43240
31
   # Business Mailboxes: # Residential Mailboxes: # of Employees:  \
0                  1,380                   27,944          28,811   
1                  1,086                   10,611          18,013   
2                    798                   17,785           9,974   
3                    398                   10,554           5,133   
4                    213                    4,364           1,198   
5                    842                   17,851           9,930   
6                    461                    5,971          15,488   
7                    489                   11,139           6,408   
8                  1,162                   19,150          18,937   
9                    530                   12,036           6,965   
10                   266                      325   

In [6]:
zip_info_df.dtypes
zip_info_df.columns

Index(['# Business Mailboxes:', '# Residential Mailboxes:', '# of Employees:',
       '113th Congressional District:', '113th Congressional Land Area:',
       '1st Quarter Payroll:', '2010 Population:', '65+:',
       'American Indian Population:', 'Annual Payroll:', 'Area Code:',
       'Asian Population:', 'Average House Value:',
       'Avg. Income Per Household:', 'Black Population:',
       'CBSA Division Name:', 'CBSA Division Population:', 'CBSA Division:',
       'CBSA Name:', 'CBSA Population:', 'CBSA Type:', 'CBSA:', 'Children:',
       'City Alias(es):', 'City State Key:', 'City Type:', 'City:',
       'Classification:', 'Counties:', 'County FIPS:', 'Current Population:',
       'Disabled Workers:', 'Division:', 'Elevation:', 'Female Median Age:',
       'Female Population:', 'Hawaiian Population:', 'Hispanic Population:',
       'Households per ZIP Code:', 'Intro Date:', 'Land Area:', 'Latitude:',
       'Longitude:', 'MSA Name:', 'MSA:', 'Male Median Age:',
       'Male P

## Location data for Hospitals and Pharmacies

Get relevant packages

In [7]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Credentials for foursquare_API

In [8]:
# @hidden_cell
CLIENT_ID = '3PRTA5P0AY2A12Z51Z1UHCZH124K5AYLU4PGQHQ0BMCV5FWD' # your Foursquare ID
CLIENT_SECRET = 'JOQYBGLRRR0QNOYE43VOVALU1DFGK4TGSR23SS3J53X2G4JW' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 60
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3PRTA5P0AY2A12Z51Z1UHCZH124K5AYLU4PGQHQ0BMCV5FWD
CLIENT_SECRET:JOQYBGLRRR0QNOYE43VOVALU1DFGK4TGSR23SS3J53X2G4JW


Information for using foursquare API 

In [9]:
zip_info_df.columns

Index(['# Business Mailboxes:', '# Residential Mailboxes:', '# of Employees:',
       '113th Congressional District:', '113th Congressional Land Area:',
       '1st Quarter Payroll:', '2010 Population:', '65+:',
       'American Indian Population:', 'Annual Payroll:', 'Area Code:',
       'Asian Population:', 'Average House Value:',
       'Avg. Income Per Household:', 'Black Population:',
       'CBSA Division Name:', 'CBSA Division Population:', 'CBSA Division:',
       'CBSA Name:', 'CBSA Population:', 'CBSA Type:', 'CBSA:', 'Children:',
       'City Alias(es):', 'City State Key:', 'City Type:', 'City:',
       'Classification:', 'Counties:', 'County FIPS:', 'Current Population:',
       'Disabled Workers:', 'Division:', 'Elevation:', 'Female Median Age:',
       'Female Population:', 'Hawaiian Population:', 'Hispanic Population:',
       'Households per ZIP Code:', 'Intro Date:', 'Land Area:', 'Latitude:',
       'Longitude:', 'MSA Name:', 'MSA:', 'Male Median Age:',
       'Male P

In [10]:
CAT_IDs = {'Hospital':'4bf58dd8d48988d196941735', 'Pharmacy': '4bf58dd8d48988d10f951735'}
geo_coords = zip_info_df[['Latitude:','Longitude:']]
radius = 10000 #about 6.2 miles

Create parameters that are not hidden

In [11]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&categoryId={}&v={}&radius={}&limit={}'
url

'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&categoryId={}&v={}&radius={}&limit={}'

Grabs data for all hospitals and pharmacies

In [12]:
i=0
# Hospital and Pharmacy Data Frame
hp_df = pd.DataFrame()
while i<len(geo_coords): 
    lat = geo_coords.iloc[i]['Latitude:']
    long = geo_coords.iloc[i]['Longitude:']
    #hospitals
    temp_url = url.format(CLIENT_ID, CLIENT_SECRET, lat, long, CAT_IDs['Hospital'], VERSION, radius, LIMIT)
    results = requests.get(temp_url).json()
    hps = results['response']['venues']
    hp_df=hp_df.append(json_normalize(hps), ignore_index=True)
    #pharmacies
    temp_url = url.format(CLIENT_ID, CLIENT_SECRET, lat, long, CAT_IDs['Pharmacy'], VERSION, radius, LIMIT)
    results = requests.get(temp_url).json()
    hps = results['response']['venues']
    hp_df=hp_df.append(json_normalize(hps), ignore_index=True)
    i+=1
hp_df

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ed664e22fb6cc941132ed13,The Village At Westerville,"[{'id': '5032891291d4c4b30a586d68', 'name': 'A...",v-1608482586,False,215 Huber Village Blvd,40.102667,-82.916989,"[{'label': 'display', 'lat': 40.102667, 'lng':...",1650,43081,US,Westerville,OH,United States,"[215 Huber Village Blvd, Westerville, OH 43081...",NaN,NaN,NaN
1,501d17b8e4b09fbdec1c12c9,H12B HEM/ONC At NCH,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1608482586,False,NaN,40.110148,-83.036035,"[{'label': 'display', 'lat': 40.11014760015202...",11751,43223,US,Columbus,OH,United States,"[Columbus, OH 43223, United States]",NaN,NaN,NaN
2,4e4523811838e44e89955e65,Heartland Hospice Serving Central Ohio,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1608482586,False,6500 Busch Blvd,40.098464,-82.992242,"[{'label': 'display', 'lat': 40.09846372, 'lng...",8058,43229,US,Columbus,OH,United States,"[6500 Busch Blvd (#210), Columbus, OH 43229, U...",#210,416270706,NaN
3,4b23ca02f964a520bf5924e3,Animal Hospital of Polaris,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",v-1608482586,False,8928 S Old State Rd,40.152349,-82.997784,"[{'label': 'display', 'lat': 40.15234859798208...",9926,43035,US,Lewis Center,OH,United States,[8928 S Old State Rd (Old State Road & Powell ...,Old State Road & Powell Road,35576025,NaN
4,4b391ed3f964a520985625e3,Mount Carmel St. Ann's Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1608482586,False,500 S Cleveland Ave,40.114484,-82.947050,"[{'label': 'display', 'lat': 40.11448386021365...",4270,43081,US,Westerville,OH,United States,"[500 S Cleveland Ave, Westerville, OH 43081, U...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,4dcd7bfe45ddbe15f870e932,Ugos Brothel,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1608482610,False,NaN,40.131222,-82.976255,"[{'label': 'display', 'lat': 40.131222, 'lng':...",1679,NaN,US,Westerville,OH,United States,"[Westerville, OH, United States]",NaN,NaN,NaN
2907,5163a42de4b095f6f027b6cc,Estra's Vitamin Shoppe,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1608482610,False,NaN,40.113375,-83.068552,"[{'label': 'display', 'lat': 40.11337511494923...",8367,NaN,US,Columbus,OH,United States,"[Columbus, OH, United States]",NaN,NaN,NaN
2908,4d2c7a8c342d6dcb7ecb1fcb,Take Care Clinic,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1608482610,False,2150 E Dublin Granville Rd,40.087143,-82.959427,"[{'label': 'display', 'lat': 40.08714294433594...",6785,43229,US,Columbus,OH,United States,"[2150 E Dublin Granville Rd, Columbus, OH 4322...",NaN,NaN,NaN
2909,4b4564fcf964a520ee0b26e3,Discount Drug Mart,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1608482610,False,8951 S Old State Rd,40.151819,-82.996391,"[{'label': 'display', 'lat': 40.15181943831064...",1535,43035,US,Lewis Center,OH,United States,"[8951 S Old State Rd, Lewis Center, OH 43035, ...",NaN,NaN,NaN


Filters to get unique Pharmacy and Hospital Locations

In [13]:
hp_df=hp_df.drop_duplicates(['id'])
hp_df

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ed664e22fb6cc941132ed13,The Village At Westerville,"[{'id': '5032891291d4c4b30a586d68', 'name': 'A...",v-1608482586,False,215 Huber Village Blvd,40.102667,-82.916989,"[{'label': 'display', 'lat': 40.102667, 'lng':...",1650,43081,US,Westerville,OH,United States,"[215 Huber Village Blvd, Westerville, OH 43081...",NaN,NaN,NaN
1,501d17b8e4b09fbdec1c12c9,H12B HEM/ONC At NCH,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1608482586,False,NaN,40.110148,-83.036035,"[{'label': 'display', 'lat': 40.11014760015202...",11751,43223,US,Columbus,OH,United States,"[Columbus, OH 43223, United States]",NaN,NaN,NaN
2,4e4523811838e44e89955e65,Heartland Hospice Serving Central Ohio,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1608482586,False,6500 Busch Blvd,40.098464,-82.992242,"[{'label': 'display', 'lat': 40.09846372, 'lng...",8058,43229,US,Columbus,OH,United States,"[6500 Busch Blvd (#210), Columbus, OH 43229, U...",#210,416270706,NaN
3,4b23ca02f964a520bf5924e3,Animal Hospital of Polaris,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",v-1608482586,False,8928 S Old State Rd,40.152349,-82.997784,"[{'label': 'display', 'lat': 40.15234859798208...",9926,43035,US,Lewis Center,OH,United States,[8928 S Old State Rd (Old State Road & Powell ...,Old State Road & Powell Road,35576025,NaN
4,4b391ed3f964a520985625e3,Mount Carmel St. Ann's Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1608482586,False,500 S Cleveland Ave,40.114484,-82.947050,"[{'label': 'display', 'lat': 40.11448386021365...",4270,43081,US,Westerville,OH,United States,"[500 S Cleveland Ave, Westerville, OH 43081, U...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2873,4ce9719f948f224baa45f65d,CVS pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1608482610,False,NaN,40.144873,-83.017583,"[{'label': 'display', 'lat': 40.14487314769656...",3200,43035,US,Lewis Center,OH,United States,"[Lewis Center, OH 43035, United States]",NaN,NaN,NaN
2882,57c9941b498e9431de05a6ef,TWC Pharmacy and Wellness Centers LLC,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1608482610,False,1980 Maxwell Ave,40.163943,-82.991883,"[{'label': 'display', 'lat': 40.163943, 'lng':...",2235,43035,US,Lewis Center,OH,United States,"[1980 Maxwell Ave, Lewis Center, OH 43035, Uni...",NaN,NaN,NaN
2902,5da621ea5e3116000881a97c,CVS Pharmacy at Target,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1608482610,False,10560 Sawmill Rd,40.143360,-83.089943,"[{'label': 'display', 'lat': 40.14336013793945...",9359,43065,US,Powell,OH,United States,"[10560 Sawmill Rd, Powell, OH 43065, United St...",NaN,NaN,NaN
2905,4c7e4fa8d598a093d551c562,County Line Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1608482610,False,484 County Line Rd,40.137418,-82.945199,"[{'label': 'display', 'lat': 40.137418, 'lng':...",3113,43081,US,Westerville,OH,United States,"[484 County Line Rd (Cleveland Ave), Westervil...",Cleveland Ave,NaN,NaN


## COVID-19 Testing Locations

Install package

In [1]:
import PyPDF2
# source of pdf scraping https://www.youtube.com/watch?v=UmPe07a3bWs

Scrapes pdf itself

In [20]:
import io

import requests
from PyPDF2 import PdfFileReader

url = 'https://www.columbus.gov/uploadedFiles/Columbus/Departments/Public_Health/All_Programs/Emergency_Preparedness/Coronavirus/LocalTesting_FQHCs_7.16.2020.pdf'

r = requests.get(url)
f = io.BytesIO(r.content)

reader = PdfFileReader(f)
contents = reader.getPage(0).extractText().split('\n')
print(contents)

['COVID-19 Community Testing', 'No one will be turned away for inability to pay. Questions? Call Columbus Public Health at 614-645-1519.', '˜˚˛˝˙ˆˇ˘ˆ', '˜', '˜', 'Information compiled by ', 'Columbus Public Health; ', 'updated 10/6/2020', 'Columbus Public Health', ' (testing provided ', 'by Mount Carmel, OhioHealth and The Ohio ', 'State University Wexner Medical Center)', '614-645-1519', 'www.columbus.gov/coronavirus', '1. Ohio Expo Center & State Fair Celeste Center ', '717 E. 17th Ave., Columbus', 'Mon.-Fri. Œ 9 a.m.-4 p.m.', 'Heart of Ohio Family Health ', '614-235-5555', 'https://heartofohiofamilyhealth.org', '2. Capital Park Family Health Center ', '2365 Innis Rd., Columbus', 'Mon., Tue., Wed., Fri. Œ 8 a.m.-5 p.m.', 'Thu., Œ 8 a.m.-7:30 p.m.', '3. Whitehall Family Health Center ', '882 S. Hamilton Rd., Whitehall', 'Mon., Wed., Thu., Fri. Œ 8 a.m.-5 p.m.', 'Tue. Œ 8 a.m.-7:30 p.m.', 'Lower Lights Christian Health Center ', '614-274-1455', 'https://llchc.org/covid19', '4. Lower Li

Filters to grab address

In [21]:
import re

address_extract ='\d+[ ](?:[A-Za-z0-9.-]+[ ]?)+(?:Avenue|Lane|Road|Boulevard|Drive|Street|Ave|Dr|Rd|Blvd|Ln|St)\.?'
p = re.compile(address_extract)
testingLocations = [ content for content in contents if p.match(content)]
p2 = re.compile('\d+\.')
covid_test_providers = [ content for content in contents if p2.match(content)]
testingLocations = testingLocations+[re.sub("\d+\.","",test) for test in covid_test_providers if re.search(address_extract,test)]
print(testingLocations)

['717 E. 17th Ave., Columbus', '2365 Innis Rd., Columbus', '882 S. Hamilton Rd., Whitehall', '777 W. State St., Columbus', '2300 W. Broad St., Columbus', '1905 Parsons Ave., Columbus', '3433 Agler Rd., Columbus', '1500 E. 17th Ave., Columbus', '750 E. Long St., Columbus', ' 86 N. Front St., Columbus']


# Get Cases by ZIP

## COVID-19 Numbers

In [26]:
zip_cases_df =  pd.read_csv("https://coronavirus.ohio.gov/static/dashboards/COVIDSummaryDataZIP.csv")
zip_cases_df.head()

,Zip Code,Population,Case Count - Cumulative,Case Count - Last 30 Days,Case Count - Last 14 Days,Case Count Per 100K - Cumulative,Case Count Per 100K - Last 30 Days,Case Count Per 100K - Last 14 Days
0,43001,"2,810",134,53,16,"4,768.70","1,886.10",569.4
1,43002,"3,744",NaN,NaN,NaN,NaN,NaN,NaN
2,43003,"3,005",107,64,21,"3,560.70","2,129.80",698.8
3,43004,"26,139","1,961",619,219,"7,502.20","2,368.10",837.8
4,43005,201,7,5,NaN,"3,482.60","2,487.60",NaN


***
#  Filtering and Modifying of data
The Dataframes are:
- zip_cases_df: Cases in Zip codes 
- testingLocations: Locations of current testing sites
- hp_df: Hospital and Pharmacy Location
- zip_info_df: Demographic Information by Zip code

***

***
#  NOT USED
***

In [14]:
import requests
from bs4 import BeautifulSoup
import json
import re
#TODO figure out how to webscrape COVID-19 data from graph
r = requests.get(
    f"https://public.tableau.com/views/COVID-19OutbreakSummary_15918845768300/COVID19Summaryp3",
     params= {
        ":embed":"y",
        ":showAppBanner":"false",
        ":showShareOptions":"true",
        ":display_count":"no",
    }
)
print(r.url)
soup = BeautifulSoup(r.text, "html.parser")

tableauData = json.loads(soup.find("textarea",{"id": "tsConfigContainer"}).text)

dataUrl = f'https://public.tableau.com{tableauData["vizql_root"]}/bootstrapSession/sessions/{tableauData["sessionid"]}'
r = requests.post(dataUrl, data= {
    "sheet_id": tableauData["sheetId"],
})
dataReg = re.search('\d+;({.*})\d+;({.*})', r.text, re.MULTILINE)
info = json.loads(dataReg.group(1))
data = json.loads(dataReg.group(2))
#print(info)
print(data["secondaryInfo"]["presModelMap"]["dataDictionary"]["presModelHolder"]["genDataDictionaryPresModel"]["dataSegments"]["0"]["dataColumns"])
#<_sre.SRE_Match object; span=(0, 109884), match='72955;{"sheetName":"DASH: Patients v discharges",>

https://public.tableau.com/views/COVID-19OutbreakSummary_15918845768300/COVID19Summaryp3?:embed=y&:showAppBanner=false&:showShareOptions=true&:display_count=no&:showVizHome=no


KeyError: 'presModelMap'

# Awesome Selenium Webscraping Action

In [23]:
from selenium import webdriver 
import time
# import Action chains  
from selenium.webdriver.common.action_chains import ActionChains 
driver = webdriver.Firefox()

# Navigate to url
driver.get("https://public.tableau.com/views/COVID-19OutbreakSummary_15918845768300/COVID19Summaryp3?%3Adisplay_count=y&%3Aorigin=viz_share_link&%3AshowVizHome=no")

# Store 'google search' button web element
time.sleep(20)
mapfre = driver.find_element_by_xpath("//*[@id=\"tabZoneId106\"]/div/div/div/div")#//*[@id=\"view8239885986803340325_964909730081303188\"]/div[1]/div[2]")
#Set x and y offset positions of element
xOffset = 0 
yOffset = 0
ActionChains(driver).move_to_element(mapfre).perform()
ActionChains(driver).move_by_offset(xOffset,yOffset).perform()
time.sleep(12)
with open('temp_covid_zip.txt', 'w', encoding='utf-8') as f:
    f.write(str(driver.page_source))

In [10]:
#this script works for extracting zip and cases range
temp = ''
with open('temp_covid_zip.txt', 'r') as fin:
    temp = fin.read()
from bs4 import BeautifulSoup
soup = BeautifulSoup(temp, 'html.parser')
table = soup.find_all('div')[len(soup.find_all('div'))-3].table #grabs the table which stores the data
print(table.find_all('tr')[0].find_all('td')[2].string)
print(table.find_all('tr')[1].find_all('td')[2].string)

43224
868 - 1289


In [12]:
#Use for Y: -39 to start off with to go from 43211 to 43224
# Navigate to url
driver.get("https://public.tableau.com/views/COVID-19OutbreakSummary_15918845768300/COVID19Summaryp3?%3Adisplay_count=y&%3Aorigin=viz_share_link&%3AshowVizHome=no")

# Store 'google search' button web element
time.sleep(20)
mapfre = driver.find_element_by_xpath("//*[@id=\"tabZoneId106\"]/div/div/div/div")#//*[@id=\"view8239885986803340325_964909730081303188\"]/div[1]/div[2]")
#Set x and y offset positions of element
xOffset = 0 
yOffset = -39
ActionChains(driver).move_to_element(mapfre).perform()
ActionChains(driver).move_by_offset(xOffset,yOffset).perform()
time.sleep(12)
with open('temp_covid_zip.txt', 'w', encoding='utf-8') as f:
    f.write(str(driver.page_source))

InvalidSessionIdException: Message: Tried to run command without establishing a connection


In [9]:
#this script works for extracting zip and cases range
# -39 is a success
temp = ''
with open('temp_covid_zip.txt', 'r') as fin:
    temp = fin.read()
from bs4 import BeautifulSoup
soup = BeautifulSoup(temp, 'html.parser')
table = soup.find_all('div')[len(soup.find_all('div'))-3].table #grabs the table which stores the data
print(table.find_all('tr')[0].find_all('td')[2].string)
print(table.find_all('tr')[1].find_all('td')[2].string)

43224
868 - 1289


In [ ]:
# @TODO: Calculate Difference between 43211 and 43224 latitude and create a scale factor